In [ ]:
from __future__ import absolute_import, division, print_function
from collections import defaultdict
import matplotlib.pyplot as plt
from chempy.chemistry import Reaction, Substance, ReactionSystem
from chempy.kinetics.ode import get_odesys
from chempy.util.graph import rsys2graph
import sympy
sympy.init_printing()
%matplotlib inline

In [ ]:
A, B, C, D = map(Substance, 'ABCD')
One = sympy.S.One
reactions = r0, r1, r2 = [
    Reaction({'A'}, {'B'}, 4*One/100, name='R1: A cons.'),
    Reaction({'B', 'C'}, {'A', 'C'}, 10**(4*One), name='R2: A reform.'),
    Reaction({'B': 2}, {'B', 'C'}, 3*10**(7*One), name='R3: C form.')
]
rsys = ReactionSystem(reactions, (A, B, C))
rsys

In [ ]:
from IPython.display import HTML
HTML(rsys.unimolecular_html_table()[0] + rsys.bimolecular_html_table()[0])

In [ ]:
rsys2graph(rsys, 'robertson.png', save='.')
from IPython.display import Image; Image('robertson.png')

In [ ]:
odesys = get_odesys(rsys, include_params=True)[0]
odesys.exprs

In [ ]:
odesys.get_jac()

In [ ]:
c0 = defaultdict(float, {'A': 1})
odesys.integrate(1e10, c0, integrator='cvode', nsteps=2000)
odesys.plot_result()
plt.gca().set_xscale('log'); plt.gca().set_yscale('log'); _ = plt.legend(loc='best')

For larger systems it is easy to loose track of what substances are actually playing a part, here the html table can help:

In [ ]:
rsys.substances['D'] = D
HTML(rsys.unimolecular_html_table()[0] + rsys.bimolecular_html_table()[0])